In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

[WDM] - Downloading: 100%|██████████| 6.78M/6.78M [00:00<00:00, 13.2MB/s]


# Srcraping tender links

The code below is used to dynamically scrape tenders for CPV codes 48000000 and 72000000.

In [7]:
# Getting search results

## navigating to the page search form
driver.get('https://ted.europa.eu/TED/search/search.do')

## geting to search results: all dates + cpv selected  codes
search_scope = driver.find_element("xpath", '//*[@id="searchScope3"]').click()
expert_search = driver.find_element("xpath", '//*[@id="switchToExpert"]').click()
search_box = driver.find_element("xpath", '//*[@id="expertSearchCriteria.query"]').send_keys("PD=[20000101 <> 20221231] AND PC=[48000000 or 72000000]")

## search click
search_button = driver.find_element("xpath", '//*[@id="scope-fs"]/button')
driver.execute_script("arguments[0].click();", search_button)


In [3]:
#getting the number of results to know how many pages to scrape
import math

results = driver.find_element('xpath','//div[@class = "pagebanner"]').text
#trim 18 spaces from the left and 7 spaces from the right of no_results
results = results[18:]
results = results[:-8]
#replace , in no. of results with nothing
results = int(results.replace(',',''))
print(results)

389714


In [ ]:
# Scraping tender links
import time
import random


for page in range(1, math.ceil(results/25)+1):
    # print page number for every 100 pages
    if page % 100 == 0:
        print("Now at page: ", page)

    # get links
    elements = driver.find_elements('xpath', '//a[@title="View this notice"]')

    # if there are not 25 elements on page, add page to error_page list
    if len(elements) != 25:
        print("Error: not 25 elements on page: ", page)
        with open('../data/error_page.txt', 'a') as f:
            f.write(str(page) + '\n')
              
    #write links to file
    for link in elements:
        with open('../data/links.txt', 'a') as f:
            f.write(link.get_attribute('href') + '\n')
        
    # write last page to file
    with open('../data/last_page.txt', 'w') as f:
        f.write(str(page))
    
    # sleep
    time.sleep(random.uniform(1,2))

    #next page
    button = driver.find_element('xpath','//div[@title="Next"]/a')
    driver.execute_script("arguments[0].click();", button) #to solve ElementClickInterceptedException

driver.quit()


In [19]:
# read links from file
with open('links.txt', 'r') as f, open('links_2.txt', 'r') as f2:
    links = f.read().splitlines()
    #read from f2 and add to links
    links.extend(f2.read().splitlines())
    f.close()

# save links in file
with open('../data//links-full.txt', 'w') as f:
    for link in links:
        f.write(link + '\n')
    f.close()

# Downloading html documents from obtained links

In [1]:
# read links from file
with open('../data/links-full.txt', 'r') as f:
    links = f.read().splitlines()
f.close()

In [2]:
len(links)

389714

In [28]:
# creating directory for scraped data
import os
os.mkdir('../data/tender-data-htmls')

In [ ]:
import os
from urllib.request import urlopen
import time
import random

# folder for saving html files
folder = '../data/tender-data-htmls//'

# saving html files
for counter, link in enumerate(links, 1): # starts from 1
    if counter < 130000: # scraping distributed across 4 machines

        # print counter to keep track of progress
        if counter % 1000 == 0:
            print("Now at link number: ", counter)

        # write last link to file 
        with open('link_counter.txt', 'w') as f:
            f.write(str(counter))
        
        # link name
        notice_id = link.split(':')[-4]
        name = folder + notice_id + ".html"
            
        # download html if file does not exist
        # if file exists, skip
        if not os.path.isfile(name):
            
            # get html
            page = urlopen(link).read().decode('utf-8')

            # save html as name
            with open(name, 'w', encoding='utf-8') as f:
                f.write(page)

        # sleep
        time.sleep(random.uniform(1,2))

        if counter % 6000 == 0:
            print("Now at link number: ", counter)
            time.sleep(600)
